Notebook for elaboration of grid frames and comparison of C results among optimizations. First we may want to import the libraries for data manipulation.

In [1]:
import os
import json
import pandas as pd
import math

# Imports for the implementation of the gif
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

And define the C results paths.

In [2]:
BASE_RESULTS = './hpc/results/'

Let's now get the files inside the base folder and list them

In [3]:
files = os.listdir(BASE_RESULTS)
print(files)

['simulation_0_1.json', 'simulation_2_3.json', '.gitkeep', 'simulation_3_4.json', 'simulation_1_2.json']


And remove the .gitkeep

In [4]:
for entry in files:
    if (entry == '.gitkeep'):
        files.remove(entry)

print(files)

['simulation_0_1.json', 'simulation_2_3.json', 'simulation_3_4.json', 'simulation_1_2.json']


Amount of simulations that were performed:

In [5]:
len(files)

4

For the moment we're considering only the first simulation

In [12]:
sim = str(BASE_RESULTS+files[3])
file = json.load(open(sim))

print(sim)

meta = file["metadata"]
data = pd.DataFrame(file["snapshots"])

./hpc/results/simulation_1_2.json


Metadata structure:

In [36]:
print(json.dumps(meta, indent=4))

{
    "time_step": 0.01,
    "ticks": 1,
    "world": {
        "width": 10,
        "height": 10,
        "width_bounds": 12,
        "height_bounds": 12
    },
    "fluid": {
        "viscosity": 0.0001,
        "density": 10,
        "diffusion": 0.0001
    }
}


Snapshots amount and structure:

In [37]:
 
print(json.dumps(file["snapshots"][0][:2], indent=2))

[
  {
    "x": 0,
    "y": 0,
    "d": 0,
    "u": 0,
    "v": 0
  },
  {
    "x": 1,
    "y": 0,
    "d": 0,
    "u": 0,
    "v": 0
  }
]


Let's now get some info from the metadata

In [28]:
#TICKS = meta
#SQUARE_BOUNDS = meta.
#print(meta[])
FRAME_POINTS = 10404

For consistency, we're also picking up the Java application output and checking whether it corresponds to the C one. This has to be done only if parameters were equal between the simulations.

In [ ]:
EQUAL_PARAMETERS = True
raw_Java = []

if EQUAL_PARAMETERS:
    raw_Java = pd.read_csv(JAVA_OUT_PATH, names=['x', 'y', 'd'])

# Now let's do the comparison
start = 0
end = FRAME_POINTS
i = 0

tmp = pd.DataFrame(columns=['x', 'y', 'd'])

# Let's sort the C output file and save it in tmp
while (start != len(raw_C)):
    
    # Take a copy of the first frame
    chunk = raw_C.loc[start:(end-1)].copy()
    
    # Sort it first along the y column and then along the x one
    chunk = chunk.sort_values(['y', 'x'], ascending=[True, True])
    
    # Append it to the temporary dataframe
    tmp = tmp.append(chunk, ignore_index = True)
   
    start = end
    end += FRAME_POINTS

tmp = tmp.reset_index(drop=True)

start = 0
end = FRAME_POINTS

# Control done for the first frame to ensure correctness between Java and C application - SLOW!!
while (i != FRAME_POINTS):
    
    java_x = tmp.x[i]
    java_y = tmp.y[i]
    
    # This call will take a while
    #java_idx = raw_Java.loc[(raw_Java['x'] == java_x) & (raw_Java['y'] == java_y) & (start <= raw_Java.index) & (raw_Java.index < end)].index[0]
    java_idx = i
    
    if not (raw_Java.d[java_idx] == tmp.d[i]):
        print("ERRORE")
        print("C index: ", i)
        print("Java index: ", java_idx)
        print(tmp.loc[i])
        print(raw_Java.loc[java_idx])
        print('\n')
        
    i += 1

# There should be a print only in case of errors.

# This shows the differencies(if any) between the two dataframes
print(tmp.compare(raw_Java))

# True if they're equal
tmp.equals(raw_Java)

Now let's put the raw data into the structure we'll be using to plot the frames; with FRAME_POINTS we can divide the output into single frames.

In [ ]:
start = 0
end = FRAME_POINTS

while (start != len(raw_C)):
    frame = {"x": raw_C.x[start:end], "y": raw_C.y[start:end], "density": raw_C.d[start:end]}
    C_data.append(frame)
    start = end
    end += FRAME_POINTS

Amount of frames:

In [ ]:
print(len(C_data))

This number includes the initial grid state plus the n ticks of the simulation.

Points per frame:

In [ ]:
print(len(C_data[0]['x']))

In [ ]:
def main():
    numframes = 51   # len(data)
    numpoints = 6724
    color_data = C_data[0]['density']  #np.random.random((numframes, numpoints))  # THIS INSTEAD DOES SOMETHING SIMILAR
    x, y, c = C_data[0]['x'], C_data[0]['y'], C_data[0]['density']

    fig = plt.figure(figsize=(10, 10))
    scat = plt.scatter(x, y, c=c, s=100)

    ani = animation.FuncAnimation(fig, update_plot, frames=range(numframes),
                                  fargs=(color_data, scat))
    plt.show()
    ani.save('animation.gif')

def update_plot(i, custom_data, scat):
    scat.set_offsets(np.c_[C_data[i]['x'], C_data[i]['y']])
    scat.set_array(C_data[i]['density'])
    return scat,

main()